In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from warnings import filterwarnings
filterwarnings("ignore")

import os
print("Orig Dir:",os.getcwd())
os.chdir('../../')
print("Current Dir:",os.getcwd())

Orig Dir: C:\Users\p.schambach\Desktop\DSR\drone_steering\models\playground
Current Dir: C:\Users\p.schambach\Desktop\DSR\drone_steering


In [20]:
from app_local.module import DataEnsembler
de = DataEnsembler(120)
de.investigate_available_datafiles(data_dir='data/gesture/', is_frame_based=True)
de.load_data()
df_orig = de.data[0]
df_orig = df_orig.iloc[17:37]
pose_df = df_orig

In [39]:
gesture_length = 2000
add_interpol_frames = 3
ms_per_frame_interpolated = 50
#pose_df.reset_index(inplace=True, drop='index')

features = list(pose_df.filter(regex = '_(x|y)$', axis = 1).columns)
t = pose_df["ms_since_start"].values

interpolators = {}
for feat in features:
    f = pose_df[feat].values
    
    cub_f = interp1d(
                t, f, 
                kind = 'cubic', 
                fill_value = (f[0],f[-1]), 
                bounds_error = False,
                assume_sorted=True
            )
    
    interpolators[feat] = cub_f
    
n = int(np.ceil(gesture_length/ms_per_frame_interpolated) + 1)
t_delta = sorted([ms_per_frame_interpolated * i for i in range(0,-n,-1)])
t_new = t_delta + t[-1]

interp_df = pd.DataFrame(columns = features)
interp_df["ms_since_start"] = t_new

for feat in features:
    interp_df[feat] = interpolators[feat](t_new)
    
interp_df

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y,ms_since_start
0,324.824346,83.344005,310.652517,248.046160,309.446652,118.768656,298.537989,53.290453,245.645253,194.604844,271.425476,244.058924,251.428286,131.827183,242.830868,248.837168,1780.0
1,323.368788,75.773507,309.535861,249.568611,309.081654,117.197528,292.177715,41.196403,244.361555,196.946440,268.890736,244.725341,251.708341,133.657546,241.582706,249.845301,1830.0
2,324.352999,70.312664,309.583477,248.486359,310.486011,117.704497,290.965351,32.351097,243.373584,199.212289,267.279564,244.843087,251.887176,134.800623,240.661150,250.377150,1880.0
3,327.274261,68.371206,310.470337,245.835951,312.702976,119.444468,295.235218,26.802849,242.802680,200.686529,266.773649,244.562769,251.638208,135.251055,240.149489,250.402850,1930.0
4,330.766877,68.600002,311.308518,243.604035,314.339271,120.690990,302.486322,20.034432,242.301325,201.407553,266.579020,244.348003,250.718284,135.720694,239.667178,250.298469,1980.0
5,333.496546,69.681234,311.250566,243.662843,314.092165,119.833709,310.333287,7.998474,241.546201,201.427565,265.950408,244.631480,248.927776,136.858443,238.864911,250.417290,2030.0
6,334.731613,70.898100,310.225769,245.688495,312.396872,117.490784,318.609876,-4.333921,240.678491,201.063871,265.077314,245.252300,246.902476,138.104916,237.992938,250.675265,2080.0
7,334.348099,72.069298,308.863223,247.418536,311.214918,116.216287,329.071035,-4.385519,240.236973,200.860467,264.952197,245.528297,245.980630,137.880246,237.810297,250.617600,2130.0
8,334.053231,72.866772,307.818675,247.634064,311.463082,116.631143,341.362119,6.211872,240.192124,201.030710,265.502552,245.135361,246.124818,136.468670,238.218683,250.425312,2180.0
9,337.487647,72.811195,307.734719,246.363111,312.825422,117.149950,352.744847,9.999261,239.875253,201.410444,265.466663,244.177749,245.765250,136.226217,238.163126,250.981744,2230.0
